In [57]:
from langchain_core.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain.chains import LLMChain
from langchain.llms import Ollama
from langchain.prompts.few_shot import FewShotPromptTemplate

class DecidArchAssistant:
    def __init__(self, configuration, system_template=None):
        self.configuration = configuration
        self.llm = self._initialize_llm()
        self.system_template = system_template

        # Examples for Few Shot Learning
        examples = [
            {
                "question": "how is the weather",
                "answer": "The weather is fine."
            },
            {
                "question": "what is the time",
                "answer": "The current time is 10 AM."
            },
        ]

        # Create example prompts in the format required by FewShotPromptTemplate
        example_prompt = PromptTemplate(
            input_variables=["question", "answer"], 
            template="Question: {question}\n{answer}"
        )
    
        self.few_shot_template = FewShotPromptTemplate(
            examples=examples,
            example_prompt=example_prompt,
            suffix="Consider the following comments about a design decision and make suggestions based on stakeholder concerns:",
            input_variables=["commento"]    
        )

        self.prompt = ChatPromptTemplate(
            messages=[
                MessagesPlaceholder("few_shot_examples"),
                HumanMessagePromptTemplate.from_template("{user_input}" + "\n" + "DecidArch: "),
            ]
        )

        self.conversation = LLMChain(
            prompt=self.prompt,
            llm=self.llm,
            verbose=True,
        )

    def _initialize_llm(self):
        # Initialize the language model here
        return Ollama()  # Adjust the initialization as per the actual requirements of Ollama
    
    def provide_suggestions(self, comment):
        # Prepare few-shot examples
        formatted_example = self.few_shot_template.format(commento=comment)
        
        # Create a list of example dictionaries
        few_shot_examples = [{"role": "assistant", "content": formatted_example}]
    
        # Use the conversation chain to get a response
        response = self.conversation.run({"user_input": comment, "few_shot_examples": few_shot_examples})
        return response

# Configuration class definition
class Configuration:
    pass

# Example usage of the class
configuration = Configuration()
assistant = DecidArchAssistant(configuration)

# Example comments
comments = [
    "I think the priority should be multiplied with the score instead of ignored when summing the final score.",
    "Yes, but when the + and - values were already given, we just looked at those."
]

# Process each comment and print the suggestions
for comment in comments:
    suggestion = assistant.provide_suggestions(comment)
    print(suggestion)



> Entering new LLMChain chain...
Prompt after formatting:
AI: Question: how is the weather
The weather is fine.

Question: what is the time
The current time is 10 AM.

Consider the following comments about a design decision and make suggestions based on stakeholder concerns:
Human: I think the priority should be multiplied with the score instead of ignored when summing the final score.
DecidArch: 

> Finished chain.
AI: Thank you for sharing your thoughts on the design decision. It's important to prioritize the needs and concerns of all stakeholders in any decision-making process. In this case, it seems that some stakeholders are advocating for multiplying the priority with the score instead of ignoring it when summing the final score.

Based on your comment, here are some suggestions to address the concerns of the stakeholders:

1. Conduct a thorough analysis: Before making any decisions, it's important to conduct a thorough analysis of the problem statement and the goals of the pro